# Contexto

Neste notebook temos o intuito de fazer previsões para uma determinada métrica.

Vamos considerar que uma empresa está observando a **Taxa de Satisfação** (%) de seus clientes desde 2022 e gostaria de projetar esses valores para 2025. Ela não possui outras variáveis que ajudem a explicar esse *KPI*, por isso, faremos a análise da série temporal.

Para tanto, vamos gerar uma base de dados aleatória e aplicar o algoritmo *SVR*.

Análise feita por Yasmin de Miranda Nobre, 15/06/2025

# Importando bibliotecas

In [91]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px

from scipy.stats import norm
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import root_mean_squared_error, mean_absolute_error
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.utils import resample
from datetime import date

# Gerando Base de Dados

In [72]:
SEED = 472393

periodo = pd.date_range(start='2022-01-01', end='2024-12-01',freq='MS')

taxa_satisfacao = norm.rvs(loc=45,scale=10, size=len(periodo))
taxa_satisfacao = np.round(taxa_satisfacao,2)

base_dados = pd.DataFrame({
   'Data' : periodo,
   'Taxa de Satisfação' : taxa_satisfacao
})


In [73]:
fig = px.line(base_dados, x="Data", y="Taxa de Satisfação")


fig.update_layout(title="Histórico da Taxa de Satisfação dos Clientes na Empresa",
    xaxis_title="",
    yaxis_title="(%)",
     font=dict(size=17))

fig.show()

Desse modo, iremos dividir a base em treino e teste, treinar e validar o modelo, e em seguida fazer as previsões para 2025.

# Treinando Modelo

Como o periodo observado é "curto" (temos apenas 2 anos de dados), vamos separar 70% da base para treinar e 30% para testar.

In [74]:
percentual_teste = 0.7

loc_ultima_data_treino = int(len(periodo)*percentual_teste)

ultima_data_treino = base_dados.iloc[loc_ultima_data_treino-1,0]
#como o python irá selecionar as linhas que vão do index 0 até loc_ultima_data_treino,
#precisamos retirar 1 unidade do loc_ultima_data_treino para saber qual será a ultima data que estará no treino

print(f"Iremos Treinar de 2022-01-01 até", ultima_data_treino.date(),f".\nIremos testar os valores de março a dezembro de 2024.")

Iremos Treinar de 2022-01-01 até 2024-01-01 .
Iremos testar os valores de março a dezembro de 2024.


In [75]:
treino = base_dados.iloc[:loc_ultima_data_treino,1]

teste = base_dados.iloc[loc_ultima_data_treino:,1]

## Redimensionando os Dados

Para utilizar o algoritmo *SVR* precisamos redimensionar os dados. Nesse sentido devemos avaliar sua distribuição e optar pelo *MinMaxScaler* ou *StandardScaler*.

Por termos gerado os dados de uma distribuição normal, já sabemos qual o comportamento da série. Asssim, iremos transformar os dados com o *StandardScaler*.

In [89]:
scaler = StandardScaler()

treino_escalado = scaler.fit_transform(treino.to_numpy().reshape(-1,1)).flatten()
teste_escalado = scaler.transform(teste.to_numpy().reshape(-1,1)).flatten()

treino_escalado

array([-0.15828151, -0.99145656, -1.53312912,  0.43560045, -0.36929451,
        0.49977451,  2.30099891,  1.9061653 ,  0.29963847,  0.53784387,
       -1.2731698 , -0.43238087, -0.74128651,  0.57373783, -0.31490972,
       -0.4519594 ,  0.57808861, -1.1154539 ,  1.91051609, -0.31164663,
       -1.11871698,  1.14804126, -0.16371999, -0.84026683, -0.37473299])

## Criando Funções de Auxilio

Iremos utilizar os dados anteriores (*lags*) como preditoras. Por isso, iremos criar uma função para qual passaremos os valores e os *lags* desejados e ela retornará o conjunto com o dado original e qual seu *lag* correspondente.

In [83]:
def criar_lags(serie, lags):
    X, y = [], []
    for i in range(max(lags), len(serie)):
        lagged_values = [serie[i - lag] for lag in lags]
        X.append(lagged_values)
        y.append(serie[i])
    return np.array(X), np.array(y)


Além disso, também criaremos uma função para testar o desempenho do *SVR* com algumas combinações de lags. A função vai retornar um *DataFrame* com as métricas *RMSE* e *MAE* de cada modelo, o que irá sugerir com quais *lags* iniciar a modelagem.

O intuito dessa função é apenas nortear a análise, por isso não será feito validação cruzada e os parametros do *SVR* já estarão definidos.

In [85]:
def testar_lags(scaler,treino_escalado,teste_escalado, combinacoes_lags):
  resultados = []
  for lags in combinacoes_lags:
      X_treino, y_treino = criar_lags(treino_escalado, lags)
      X_teste, y_teste = criar_lags(teste_escalado, lags)

      # Treinar o modelo SVR
      model = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=0.01)
      model.fit(X_treino, y_treino)

      # Previsões
      y_pred = model.predict(X_teste) #previsoes ficam em outra escala, a que usamos com StandardScaler
      y_pred_original = scaler.inverse_transform(y_pred.reshape(-1, 1))
      y_teste_original = scaler.inverse_transform(y_teste.reshape(-1, 1))

      # Avaliação
      rmse = root_mean_squared_error(y_teste_original, y_pred_original)
      mae = mean_absolute_error(y_teste_original, y_pred_original)

      resultados.append({
          'Lags': lags,
          'RMSE': rmse,
          'MAE': mae
        })

  df_resultados = pd.DataFrame(resultados)
  df_resultados = df_resultados.sort_values(by='RMSE').reset_index(drop=True)
  return df_resultados

Mais a frente será aplicado o *fit* do GridSearchCV para otimizar os parâmetros do *SVR*. Por isso, iremos deixar aqui alguns objetos e aplicar o *fit* mais a frente.

In [95]:
tscv = TimeSeriesSplit(n_splits=5)

pipeline = Pipeline([
  ('scaler',scaler),
  ('svr', SVR())
])

param_grid = {
  'svr__C':[0.1,1,10,50,100],
  'svr__gamma':['scale','auto',0.01,0.1,1],
  'svr__epsilon':[0.001,0.01,0.1,0.2]
}

grid_search = GridSearchCV(
  estimator= pipeline,
  param_grid= param_grid,
  cv= tscv,
  scoring = 'neg_root_mean_squared_error',
  n_jobs= -1
)


## Análise Inicial

In [93]:
combinacoes = [
    [1,3,6],
    [1,3,9],
    [1,4,6],
    [1,4,9],
    [1,6,9],
    [2,4,6],
    [2,4,9]
]
#aqui, basicamente estamos considerando quantos meses atrás estariam influenciando os dados
#exemplo: a taxa de satisfação de janeiro será explicada pela de dezembro (mes anterior,1), outubro (3) e julho (6)

resultados = testar_lags(scaler, treino_escalado, teste_escalado, combinacoes)
resultados

,Lags,RMSE,MAE
0,"[2, 4, 9]",5.181174,5.144802
1,"[1, 6, 9]",8.909099,7.095329
2,"[2, 4, 6]",9.524200,7.405467
3,"[1, 4, 9]",12.677116,10.648118
4,"[1, 4, 6]",14.666517,9.944304
5,"[1, 3, 6]",16.856462,12.402350
6,"[1, 3, 9]",17.726296,15.655910


### Testando Lags 2,4,9

In [97]:
lags_satisfacao = [2,4,9]

X_treino_satisfacao, y_treino_satisfacao = criar_lags(treino_escalado, lags_satisfacao)
X_teste_satisfacao, y_teste_satisfacao = criar_lags(teste_escalado, lags_satisfacao)


grid_search.fit(X_treino_satisfacao,y_treino_satisfacao)


modelo_satisfacao = SVR(kernel='rbf', C=grid_search.best_params_.get('svr__C'),
                        gamma= grid_search.best_params_.get('svr__gamma'),
                        epsilon=grid_search.best_params_.get('svr__epsilon'))
modelo_satisfacao.fit(X_treino_satisfacao,y_treino_satisfacao)

SVR(C=1, epsilon=0.001, gamma=0.01)